In [ ]:
import pandas as pd
import json
import numpy as np

with open('./result/origin_val_result.json') as f:
    data = json.load(f)

In [3]:
def smape_loss(pred, true):
    # assume pred and true are numpy arrays sized (n,)
    return np.mean(np.abs(pred - true) / (np.abs(pred) + np.abs(true)))

In [4]:
records_loss = []
batch_size = 2
for record in data:
    device_batch, pred_batch, true_batch = record['device'], record['pred'], record['true']

    for index in range(batch_size):
        device = device_batch[index]
        pred = pred_batch[index]
        true = true_batch[index]
        # pred, true: list shaped (96,2), each element is a list of [cpu, mem]
        cpu_pred = np.array(pred)[:, 0]
        mem_pred = np.array(pred)[:, 1]
        cpu_true = np.array(true)[:, 0]
        mem_true = np.array(true)[:, 1]

        cpu_smape = smape_loss(cpu_pred, cpu_true)
        mem_smape = smape_loss(mem_pred, mem_true)
        cpu_mae = np.mean(np.abs(cpu_pred - cpu_true))
        mem_mae = np.mean(np.abs(mem_pred - mem_true))
        cpu_mse = np.mean((cpu_pred - cpu_true) ** 2)
        mem_mse = np.mean((mem_pred - mem_true) ** 2)

        records_loss.append({
            'device': device,
            'cpu_smape': cpu_smape,
            'cpu_mae': cpu_mae,
            'cpu_mse': cpu_mse,
            'mem_smape': mem_smape,
            'mem_mae': mem_mae,
            'mem_mse': mem_mse
        })

df = pd.DataFrame(records_loss)


In [5]:
# df.to_csv('pred_result_2025-01-23.csv', index=False)
# count smape loss levels: 0.01, 0.02, ... , 0.19, 0.20, 0.30, 0.40, 0.50, others
smape_levels = [0.01 * i for i in range(20)] + [0.20, 0.30, 0.40, 0.50]
cpu_smape_counts = []
mem_smape_counts = []
for level in smape_levels:
    cpu_smape_counts.append(df[df['cpu_smape'] <= level].shape[0])
    mem_smape_counts.append(df[df['mem_smape'] <= level].shape[0])

cpu_smape_counts.append(df.shape[0])
mem_smape_counts.append(df.shape[0])

# erase the first element of smape_levels
df_cpu = pd.DataFrame({
    'smape_level': smape_levels[1:] + ['>0.50'],
    'count': np.diff(cpu_smape_counts),
    'count_ratio': np.diff(cpu_smape_counts) / df.shape[0],
    'count_cumsum': cpu_smape_counts[1:],
    'sum_ratio': np.array(cpu_smape_counts[1:]) / df.shape[0]
})
df_mem = pd.DataFrame({
    'smape_level': smape_levels[1:] + ['>0.50'],
    'count': np.diff(mem_smape_counts),
    'count_ratio': np.diff(mem_smape_counts) / df.shape[0],
    'count_cumsum': mem_smape_counts[1:],
    'sum_ratio': np.array(mem_smape_counts[1:]) / df.shape[0]
})
df_cpu['count_ratio'] = df_cpu['count_ratio'].map(lambda x: '{:.2%}'.format(x))
df_mem['count_ratio'] = df_mem['count_ratio'].map(lambda x: '{:.2%}'.format(x))
df_cpu['sum_ratio'] = df_cpu['sum_ratio'].map(lambda x: '{:.2%}'.format(x))
df_mem['sum_ratio'] = df_mem['sum_ratio'].map(lambda x: '{:.2%}'.format(x))

In [6]:
print(df_cpu)

   smape_level  count count_ratio  count_cumsum sum_ratio
0         0.01     10       0.38%            10     0.38%
1         0.02    227       8.72%           237     9.10%
2         0.03    509      19.55%           746    28.65%
3         0.04    354      13.59%          1100    42.24%
4         0.05    154       5.91%          1254    48.16%
5         0.06    139       5.34%          1393    53.49%
6         0.07    153       5.88%          1546    59.37%
7         0.08    126       4.84%          1672    64.21%
8         0.09     91       3.49%          1763    67.70%
9          0.1     57       2.19%          1820    69.89%
10        0.11     37       1.42%          1857    71.31%
11        0.12     31       1.19%          1888    72.50%
12        0.13     17       0.65%          1905    73.16%
13        0.14     19       0.73%          1924    73.89%
14        0.15     31       1.19%          1955    75.08%
15        0.16     56       2.15%          2011    77.23%
16        0.17

In [7]:
df.describe()

,cpu_smape,cpu_mae,cpu_mse,mem_smape,mem_mae,mem_mse
count,2604.000000,2604.000000,2604.000000,2604.000000,2604.000000,2604.000000
mean,0.091939,2.393919,43.803122,0.009294,0.676331,9.824807
std,0.093390,4.497810,180.817737,0.033745,2.752718,75.650978
min,0.005339,0.090180,0.015345,0.000011,0.000785,0.000001
25%,0.027765,0.602998,1.196840,0.000759,0.054076,0.004325
50%,0.053324,1.011930,3.841170,0.001551,0.096499,0.013156
75%,0.149412,2.362707,13.836604,0.003661,0.236786,0.073627
max,0.710397,48.680144,2784.626632,0.429166,36.494826,1540.609896
